## Notebook to get Model1 Predictions

In [9]:
# Import Libraries

import pandas as pd
# import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
# import glob
# import re
import os

# import nltk 
# #nltk.download('stopwords')
# #nltk.download('wordnet')
# from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer

# from wordcloud import WordCloud

# from PIL import Image

# from sklearn.model_selection import train_test_split

# from keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences

# import gensim.models.keyedvectors as word2vec

# from tensorflow.keras.layers import Embedding,Dense,LSTM,Bidirectional
# from tensorflow.keras.layers import BatchNormalization,Dropout

# from tensorflow.keras import Sequential

import pickle
from keras.models import load_model

In [2]:
# Read input CSV file with scrapped tweets
path = r'Tweet_All.csv'
tweet_all = pd.read_csv(path)

In [3]:
# dropping duplicates in the dataframe
tweet_all = tweet_all.drop_duplicates(subset=['Text'], keep='first').reset_index(drop=True)
tweet_all

,Username_Query,Datetime,Tweet Id,Text
0,kevinhart4real,2021-01-22 02:41:31,1.352446e+18,😂😂😂😂😂😂 https://t.co/S6bufs38c6
1,kevinhart4real,2021-01-21 21:00:42,1.352360e+18,Mood on this beautiful Thursday.... #LiveLoveL...
2,kevinhart4real,2021-01-21 16:47:20,1.352297e+18,Just posted a photo https://t.co/QA4YrxAD39
3,kevinhart4real,2021-01-20 01:18:07,1.351701e+18,For Celebrity Game Face tonight: \n\nThese cou...
4,kevinhart4real,2021-01-19 21:20:56,1.351641e+18,Haters will say that I’m not tall..... 🤷🏾‍♂️ s...
...,...,...,...,...
21558,riceuniversity,2019-07-18 15:49:29,1.151882e+18,RT @ScottESolomon: Great to see this at @Moody...
21559,riceuniversity,2019-07-18 15:08:59,1.151871e+18,@scifiya7th @RiceU_STEM @conocophillips Yes of...
21560,riceuniversity,2019-07-18 14:24:17,1.151860e+18,Space veterans who worked on Apollo 11 joined ...
21561,riceuniversity,2019-07-18 14:04:11,1.151855e+18,RT @NASA_Johnson: President Kennedy’s challeng...


In [4]:
# loading model
model = load_model('/Users/user/Desktop/Project3/model/model1.h5')
print("Model1 Loaded")
file = open('/Users/user/Desktop/Project3/model/tokenizer1.pickle', 'rb')
tokens = pickle.load(file)
file.close()
print("Tokenizer1 Loaded")

Model1 Loaded
Tokenizer1 Loaded


In [5]:
# read "text" column from all scrapped tweets or enter a custom phrase
text = list(tweet_all.Text)[0:10]
#text = ['I love spring', 'I have winter']

#### Model Prediction

In [14]:
# Define score labels and model prediction functions
def decode_sentiment(score):
    label = None
    if score <= 0.5:
        label = 'Negative'
    else:
        label = 'Positive'
    
    return label

def predict(text):
    x_test = pad_sequences(tokens.texts_to_sequences([text]), maxlen=max_length,padding=padd_type,truncating=trunc_type)
    score = model.predict([x_test])
    label = decode_sentiment(score)
    
    return  {"text": text, "label": label, "score": float(score)}